In [ ]:
import os
import dotenv

dotenv.load_dotenv()
MAIN_SAFE = "0x4c190e3C582e877942007Db3009feACc1Da6C481"
METAMASK_EOA = os.getenv("WALLET_ADDRESS")
POLYMARKET_SAFE = "0x2617746e3Da3cC26EeD139d873f0A05137023e40"
TRADER_EOA_PRIVATE_KEY = os.getenv("TRADER_EOA_PRIVATE_KEY")
PRIVATE_KEY = os.getenv("PRIVATE_KEY")

ACC_2_ADDRESS = "0xfd4fac895d20912b5d3abdd8695d891baa5e90d5"
ACC_2_PRIVATE_KEY = os.getenv("ACC_2_PRIVATE_KEY")
USDCE_ADDRESS = "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174"

POLYGON_RPC = os.getenv("POLYGON_RPC")  # Polygon rpc url

In [ ]:
Web3.to_checksum_address(METAMASK_EOA)

In [ ]:
SAFE_ABI = [
    {
        "inputs": [
            {"internalType": "bytes32", "name": "hashToApprove", "type": "bytes32"}
        ],
        "name": "approveHash",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function",
    },
    {
        "inputs": [
            {"internalType": "address", "name": "to", "type": "address"},
            {"internalType": "uint256", "name": "value", "type": "uint256"},
            {"internalType": "bytes", "name": "data", "type": "bytes"},
            {"internalType": "uint8", "name": "operation", "type": "uint8"},
            {"internalType": "uint256", "name": "safeTxGas", "type": "uint256"},
            {"internalType": "uint256", "name": "baseGas", "type": "uint256"},
            {"internalType": "uint256", "name": "gasPrice", "type": "uint256"},
            {"internalType": "address", "name": "gasToken", "type": "address"},
            {"internalType": "address", "name": "refundReceiver", "type": "address"},
            {"internalType": "bytes", "name": "signatures", "type": "bytes"},
        ],
        "name": "execTransaction",
        "outputs": [{"internalType": "bool", "name": "success", "type": "bool"}],
        "stateMutability": "payable",
        "type": "function",
    },
    {
        "inputs": [],
        "name": "nonce",
        "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "stateMutability": "view",
        "type": "function",
    },
]

In [ ]:
from web3 import Web3
from eth_account import Account
import eth_abi
from eth_account.messages import encode_defunct

# ---------------------------------------------------------
# CONFIG
# ---------------------------------------------------------

RPC = POLYGON_RPC
w3 = Web3(Web3.HTTPProvider(RPC))

SAFE_A = Web3.to_checksum_address(POLYMARKET_SAFE)
SAFE_B = Web3.to_checksum_address(MAIN_SAFE)
WALLET_PK = PRIVATE_KEY

RECIPIENT = TO = Web3.to_checksum_address(METAMASK_EOA)
VALUE = w3.to_wei(1, "ether")  # native token amount
OPERATION = 0  # CALL


# ---------------------------------------------------------
# CONTRACT INSTANCES
# ---------------------------------------------------------

safeA = w3.eth.contract(address=SAFE_A, abi=SAFE_ABI)
safeB = w3.eth.contract(address=SAFE_B, abi=SAFE_ABI)

acct = Account.from_key(WALLET_PK)


# ---------------------------------------------------------
# STEP— setup values
# ---------------------------------------------------------

# USDC.e on Polygon
USDCe = Web3.to_checksum_address(USDCE_ADDRESS)
AMOUNT = 10_000  # 0.01 USDC.e (6 decimals)

ZERO = Web3.to_checksum_address("0x0000000000000000000000000000000000000000")


# ---------------------------------------------------------
# MINIMAL SAFE ABI (approveHash + execTransaction + nonce)
# ---------------------------------------------------------
SAFE_ABI = [
    {
        "inputs": [
            {"internalType": "bytes32", "name": "hashToApprove", "type": "bytes32"}
        ],
        "name": "approveHash",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function",
    },
    {
        "inputs": [],
        "name": "nonce",
        "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "stateMutability": "view",
        "type": "function",
    },
    {
        "inputs": [
            {"internalType": "address", "name": "to", "type": "address"},
            {"internalType": "uint256", "name": "value", "type": "uint256"},
            {"internalType": "bytes", "name": "data", "type": "bytes"},
            {"internalType": "uint8", "name": "operation", "type": "uint8"},
            {"internalType": "uint256", "name": "safeTxGas", "type": "uint256"},
            {"internalType": "uint256", "name": "baseGas", "type": "uint256"},
            {"internalType": "uint256", "name": "gasPrice", "type": "uint256"},
            {"internalType": "address", "name": "gasToken", "type": "address"},
            {"internalType": "address", "name": "refundReceiver", "type": "address"},
            {"internalType": "bytes", "name": "signatures", "type": "bytes"},
        ],
        "name": "execTransaction",
        "outputs": [{"internalType": "bool", "name": "success", "type": "bool"}],
        "stateMutability": "payable",
        "type": "function",
    },
]


# ---------------------------------------------------------
# CONTRACTS + WALLET
# ---------------------------------------------------------
safeA = w3.eth.contract(address=SAFE_A, abi=SAFE_ABI)
safeB = w3.eth.contract(address=SAFE_B, abi=SAFE_ABI)
wallet = Account.from_key(WALLET_PK)


# ---------------------------------------------------------
# STEP 1 — BUILD THE ERC20 TRANSFER() CALL
# ---------------------------------------------------------
selector = Web3.keccak(text="transfer(address,uint256)")[:4]
data = selector + eth_abi.encode(["address", "uint256"], [RECIPIENT, AMOUNT])


# ---------------------------------------------------------
# STEP 2 — BUILD SAFE A TX HASH
# ---------------------------------------------------------
nonceA = safeA.functions.nonce().call()

tx_encoded = eth_abi.encode(
    [
        "address",
        "uint256",
        "bytes",
        "uint8",
        "uint256",
        "uint256",
        "uint256",
        "address",
        "address",
        "uint256",
    ],
    [
        USDCe,  # to
        0,  # value = 0 (ERC20 transfer)
        data,  # data = encoded transfer()
        0,  # operation CALL
        0,  # safeTxGas
        0,  # baseGas
        0,  # gasPrice
        ZERO,
        ZERO,
        nonceA,
    ],
)

safeA_hash = w3.keccak(
    b"\x19\x01"
    + b"\x00" * 32  # domain separator (Safe L1/L2 compatible)
    + w3.keccak(tx_encoded)
)

print("Safe A tx hash:", safeA_hash.hex())


# ---------------------------------------------------------
# STEP 3 — WALLET SIGNS AS OWNER OF SAFE B
# ---------------------------------------------------------
from eth_account.messages import encode_defunct

msg = encode_defunct(primitive=safeA_hash)
signed = wallet.sign_message(msg)

sig_bytes = (
    signed.r.to_bytes(32, "big") +
    signed.s.to_bytes(32, "big") +
    bytes([signed.v])
)

print("Wallet → SafeB signature:", sig_bytes.hex())


# ---------------------------------------------------------
# STEP 4 — SAFE B APPROVES HASH ON-CHAIN
# ---------------------------------------------------------
tx1 = safeB.functions.approveHash(safeA_hash).build_transaction(
    {
        "from": wallet.address,
        "nonce": w3.eth.get_transaction_count(wallet.address),
        "gas": 150_000,
        "gasPrice": w3.eth.gas_price,
    }
)

signed1 = wallet.sign_transaction(tx1)
tx_hash1 = w3.eth.send_raw_transaction(signed1.raw_transaction)
print("approveHash tx:", tx_hash1.hex())

receipt1 = w3.eth.wait_for_transaction_receipt(tx_hash1)
print("approveHash status:", receipt1.status)


# ---------------------------------------------------------
# STEP 5 — SAFE A EXECUTES TRANSACTION
# ---------------------------------------------------------
# Signature blob: only Safe B signing → one 65-byte signature
signatures = sig_bytes

tx2 = safeA.functions.execTransaction(
    USDCe,
    0,
    data,
    0,  # CALL
    0,  # safeTxGas
    0,  # baseGas
    0,  # gasPrice
    ZERO,
    ZERO,
    signatures,
).build_transaction(
    {
        "from": wallet.address,  # whoever pays gas
        "nonce": w3.eth.get_transaction_count(wallet.address),
        "gas": 200_000,
        "gasPrice": w3.eth.gas_price,
    }
)

signed2 = wallet.sign_transaction(tx2)
tx_hash2 = w3.eth.send_raw_transaction(signed2.raw_transaction)
print("execTransaction tx:", tx_hash2.hex())

receipt2 = w3.eth.wait_for_transaction_receipt(tx_hash2)
print("execTransaction status:", receipt2.status)